# Introduction

- Previous: [01-ds-submit-project.ipynb](./01-ds-submit-project.ipynb)
- Next: [03-ds-execute-code.ipynb](./03-ds-execute-code.ipynb)

# Prerequisites
You have ran the [01-ds-submit-project.ipynb](./01-ds-submit-project.ipynb) and have a pending project and code request from the DS in both `canada-domain` and `italy-domain`.

In [ ]:
# syft absolute
import syft as sy

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082

## Launch Domains

In [ ]:
# Launch the domain nodes we setup in the previous notebook
canada_node = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True
)

italy_node = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True
)

# Log in to the first domain node as the data owner

In [ ]:
do_canada_client = canada_node.login(email="info@openmined.org", password="changethis")

# View all pending project requests

In [ ]:
do_canada_client.projects

In [ ]:
# Select the project you want to work with
canada_project = do_canada_client.projects[0]
canada_project

In [ ]:
# Select a request and explore its attributes
request = canada_project.requests[0]
request

In [ ]:
canada_project.code[0]

In [ ]:
canada_project.code[0].status(canada_project, verbose=True)

# Review the code, dataset and policies

In [ ]:
# See the code written by the Data Scientist and its metadata in the request
func = request.code
func

In [ ]:
# Find the assets required for the computation
func.assets

In [ ]:
# Review the deployment policy
assert func.deployment_policy_type == sy.RunOnEnclave
deployment_provider = func.deployment_policy_init_kwargs["provider"]
deployment_provider

In [ ]:
# Execute the code with mock data
# TODO this should take mock data from all dependent domains and execute the function
# func.execute_with_mock_data()

# Approve the request

In [ ]:
request.approve()

## Check Project Status

In [ ]:
canada_project.sync()

In [ ]:
canada_project.code[0].status(canada_project, verbose=True)

# Approve the code from the other domain node

In [ ]:
do_italy_client = italy_node.login(email="info@openmined.org", password="changethis")

In [ ]:
italy_project = do_italy_client.projects[0]

In [ ]:
italy_project.code[0].status(italy_project, verbose=True)

In [ ]:
requests = italy_project.requests
assert len(requests) == 1
request = requests[0]

func = request.code
# TODO change below to 2 once we start showing assets from other domains
assert len(func.assets) == 1
assert func.deployment_policy_type == sy.RunOnEnclave

request.approve()

## Get Code Status on Project

In [ ]:
italy_project.sync()

In [ ]:
italy_project.code[0].status(italy_project, verbose=True)

In [ ]:
canada_project.sync()

In [ ]:
canada_project.code[0].status(canada_project, verbose=True)

# Cleanup local domain servers

In [ ]:
if canada_node.deployment_type.value == "python":
    canada_node.land()

if italy_node.deployment_type.value == "python":
    italy_node.land()